# Adapter

参考:
- https://github.com/faif/python-patterns/blob/master/structural/adapter.py
- http://design-patterns.readthedocs.io/zh_CN/latest/structural_patterns/adapter.html#

适配器模式包含如下角色:
- Target: 目标抽象类
- Adapter: 适配器类
- Adaptee: 适配者类
- Client: 客户类

## 示例1: 对象适配器

In [1]:
# Adaptee

class Cat(object):
    def __init__(self):
        self.name = 'Cat'

    def meow(self):
        return 'meow...'


class Dog(object):
    def __init__(self):
        self.name = 'Dog'

    def bark(self):
        return 'woof...'


class Human(object):
    def __init__(self):
        self.name = 'Human'

    def speak(self):
        return 'hello...'

# Adapter

class Adapter(object):
    
    def __init__(self, obj, **adapted_methods):
        self.obj = obj
        self.__dict__.update(adapted_methods)
    
    def __getattr__(self, attr):
        return getattr(self.obj, attr)

In [2]:
dog = Dog()
cat = Cat()
you = Human()

objs = [
    Adapter(dog, hello=dog.bark),
    Adapter(cat, hello=cat.meow),
    Adapter(you, hello=you.speak),
]

for obj in objs:
    print obj.name, obj.hello()

Dog woof...
Cat meow...
Human hello...


## 示例2: 对象适配器

- https://gist.github.com/pazdera/1145859

In [6]:
# Adaptee interface

class EuropeanSocketInterface:
    def voltage(self): pass
    def live(self): pass
    def neutral(self): pass
    def earth(self): pass


# Adaptee

class Socket(EuropeanSocketInterface):
    def voltage(self):
        return 230
    
    def live(self):
        return 1
    
    def neutral(self):
        return -1
    
    def earth(self):
        return 0


# Target Interface

class ChinaSocketInterface:
    def voltage(self): pass
    def live(self): pass
    def neutral(self): pass


# Adapter

class Adapter(ChinaSocketInterface):
    __socket = None
    
    def __init__(self, socket):
        self.__socket = socket
    
    def voltage(self):
        return 220
    
    def live(self):
        return self.__socket.live()
    
    def neutral(self):
        return self.__socket.neutral()


# Client

class Computer:
    __power = None
    
    def __init__(self, power):
        self.__power = power
    
    def turn_on(self):
        if self.__power.voltage() > 220:
            print 'Voltage is too high...'
        else:
            if self.__power.live() == 1 and self.__power.neutral() == -1:
                print 'Coffee time!'
            else:
                print 'Turning on...'

In [9]:
socket = Socket()
adapter = Adapter(socket)
computer = Computer(adapter)

computer.turn_on()

Coffee time!


## 示例3: 类适配器 & 对象适配器

- http://ginstrom.com/scribbles/2009/03/27/the-adapter-pattern-in-python/

In [17]:
# Target

class Creature(object):
    
    def make_noise(self):
        raise NotImplemented


class Person(Creature):
    def __init__(self, name):
        self.name = name
    
    def make_noise(self):
        return "Hello"


class Dog(object):
    def __init__(self, name):
        self.name = name
    
    def bark(self):
        return "Woof"


# 类适配器

class DogClassAdapter(Creature, Dog):
    def __init__(self, name):
        Dog.__init__(self, name)  # <==
    
    def make_noise(self):
        return self.bark()


# 对象适配器

class DogObjectAdapter(Creature):
    def __init__(self, dog):
        self.dog = dog
    
    def make_noise(self):
        return self.dog.bark()
    
    def __getattr__(self, attr):
        return getattr(self.dog, attr)

In [20]:
person = Person('abc')
print person.make_noise()

dog = DogObjectAdapter(Dog('def'))
print dog.make_noise()

Hello
Woof


### 不适用Target, 使用鸭子模型

In [32]:
class Person(object):
    def __init__(self, name):
        self.name = name
    
    def make_noise(self):
        return 'Hello'


class Dog(object):
    def __init__(self, name):
        self.name = name
    
    def bark(self):
        return "Woof"


class DogAdapter(object):
    def __init__(self, dog):
        self.dog = dog
    
    def make_noise(self):
        return self.dog.bark()
    
    def __getattr__(self, item):
        return getattr(self.dog, item)

In [33]:
person = Person('person')
dog = DogAdapter(Dog('dog'))

print person.make_noise()
print dog.make_noise()

Hello
Woof


### 使用通用的适配器, 参考示例1